In [ ]:
import os
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

load_dotenv()

DB_PARAMS = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

In [ ]:
def get_reviews_df():
    conn = psycopg2.connect(**DB_PARAMS)
    query = """
    SELECT b.bank_name, r.review_text, r.rating, r.sentiment_label, r.sentiment_score, r.review_date
    FROM reviews r
    JOIN banks b ON r.bank_id = b.bank_id;
    """
    df = pd.read_sql(query, conn)
    conn.close()
    return df


df = get_reviews_df()

In [ ]:
print("### Data Overview")
print(f"Total reviews: {len(df)}")
print("Reviews per bank:")
print(df['bank_name'].value_counts())
print("\nSample data:")
df.head()

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(data=df, x='rating', hue='bank_name', palette='Set2')
plt.title('Rating Distribution by Bank')
plt.xlabel('Rating (Stars)')
plt.ylabel('Number of Reviews')
plt.legend(title='Bank')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(data=df, x='sentiment_label', hue='bank_name', order=['positive', 'neutral', 'negative'], palette='Set1')
plt.title('Sentiment Distribution by Bank')
plt.xlabel('Sentiment')
plt.ylabel('Number of Reviews')
plt.legend(title='Bank')
plt.show()

In [ ]:
avg_sentiment = df.groupby('bank_name')['sentiment_score'].mean().sort_values(ascending=False)
print("### Average Sentiment Score per Bank:")
print(avg_sentiment)